## 사용된 것
* 0. 로지스틱회귀를 클래스로 구현해서 실행
* 1. Class 생성 및 활용(__init__, def)
* 2. 범주형(문자열) -> 연속형(0 또는 1) 변환
* 3. pred == actual 퍼센트 출력
* 4. dataset 크기가 작아서(400줄) 데이터셋과 테스트셋을 어떻게 나누느냐에 따라 크게 차이가 난다.
* 5. 앞으로는 sklearn split 함수를 쓰는 습관을 들이자

In [1]:
import numpy as np
from sklearn import datasets
import pandas as pd

#GD를 활용한 LogisticRegression
class LogisticRegression:
    def __init__(self, learning_rate=0.00001, threshold=0.000001, max_iterations=100000, fit_intercept=True, verbose=False):
        self._learning_rate = learning_rate  # 학습 계수
        self._max_iterations = max_iterations  # 반복 횟수
        self._threshold = threshold  # 학습 중단 계수
        self._fit_intercept = fit_intercept  # 절편 사용 여부를 결정
        self._verbose = verbose  # 중간 진행사항 출력 여부

    # theta(W) 계수들 return
    def get_coeff(self):
        return self._W

    # 절편 추가
    def add_intercept(self, x_data): # intercept. feature 외 변수.
        intercept = np.ones((x_data.shape[0], 1))
        return np.concatenate((intercept, x_data), axis=1)

    # 시그모이드 함수(로지스틱 함수)
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def cost(self, h, y):##어떤뜻인지 파악하기(cost function에 대한 이해필요(GD,SGD 찾아보기)) * Log Likelihood의 음수는 목적함수. lo
        ##  cost = Sum(  -y*log(sigmoid(Wx+b))   -   (1-y)*log(1-(sigmoid(Wx+b))) ) /n
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    
    def fit(self, x_data, y_data):
        num_examples, num_features = np.shape(x_data)

        if self._fit_intercept:
            x_data = self.add_intercept(x_data)

        
        self._W = np.zeros(x_data.shape[1])

        for i in range(self._max_iterations):
            z = np.dot(x_data, self._W) # 독립변수(x_data)와 계수(coeff, _W)를 내적한다
            hypothesis = self.sigmoid(z) # 내적한 결과를 시그모이드 함수(로지스틱 함수)에 넣어 음의 무한대와 양의 무한대 사이의 살수에서 0과 1사이의 실수로 변환하여 y_hat(hypothesis)를 도출한다.

            #실제값과 예측값의 차이
            diff = hypothesis - y_data
            cost = self.cost(hypothesis, y_data)

            ##어떤 과정인지 설명하기 * 우도 함수에 로그를 씌운 함수의 음수는 목적함수와 같다. 
            ## cost 함수의 편미분 : transposed X * diff / n
            gradient = np.dot(x_data.transpose(), diff) / num_examples
            self._W -= self._learning_rate * gradient

            if cost < self._threshold:
                return False
           
            if (self._verbose == True and i % 100 == 0):
                print('cost :', cost)

    def predict_prob(self, x_data):
        if self._fit_intercept:
            x_data = self.add_intercept(x_data)

        return self.sigmoid(np.dot(x_data, self._W))

    def predict(self, x_data):
        
        return self.predict_prob(x_data).round()##왜 라운드 함수를 쓰는지 * 0과 1사이의 실수를 0 혹은 1로 분류하기 위해서다.
    ## 0또는 1로 분류(반올림 함수)

In [2]:
df  = pd.read_csv('Social_Network_Ads.csv')
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [3]:
df['Gender']=df['Gender'].replace('Male',0)
df['Gender']=df['Gender'].replace('Female',1)

In [4]:
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,0,19,19000,0
1,15810944,0,35,20000,0
2,15668575,1,26,43000,0
3,15603246,1,27,57000,0
4,15804002,0,19,76000,0


In [5]:
df_X = df[['Gender', 'Age', 'EstimatedSalary']]

In [6]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler()
# df_x = sc.fit_transform(df_X)

In [7]:
df_y=df['Purchased']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df_X,df_y, test_size=0.2, random_state=42)

In [10]:
y_train

3      0
18     1
202    1
250    0
274    1
63     1
248    0
301    1
108    0
90     0
233    1
335    0
118    0
220    0
180    0
314    0
373    1
380    0
239    1
75     1
247    1
110    0
16     1
66     0
153    0
7      1
19     1
137    1
355    1
349    0
      ..
344    1
235    1
252    1
21     1
313    1
160    1
276    0
191    0
293    0
343    1
257    0
308    1
149    0
130    0
151    0
359    0
99     0
372    0
87     0
330    0
214    0
121    0
399    1
20     1
188    0
71     0
106    0
270    0
348    0
102    0
Name: Purchased, Length: 320, dtype: int64

In [11]:
# X_train = df_X.iloc[0:320,:]
# X_test = df_X.iloc[321:401,:]
# y_train = df_y.iloc[0:320,:]
# y_test = df_y.iloc[321:401,:]

In [12]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler()

In [13]:
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [14]:
# y_train = y_train['Purchased']
# y_test = y_test['Purchased']

In [15]:
X_train.head()

,Gender,Age,EstimatedSalary
3,1,27,57000
18,0,46,28000
202,1,39,134000
250,1,44,39000
274,1,57,26000


In [16]:
X_train.shape, y_train.shape

((320, 3), (320,))

In [17]:
a = LogisticRegression()
a.fit(X_train, y_train)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log


In [18]:
a.predict_prob(X_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
a.predict_prob(X_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
pre = a.predict(X_test)

In [21]:
count = 0
for i in range(len(pre)):
    if pre[i]==np.array(y_test)[i]:
        count+=1
print((count/len(pre))*100, '%')

65.0 %
